In [ ]:
# source of new datasets.
# # https://www.cso.ie/en/releasesandpublications/ep/p-dcmec/datacentresmeteredelectricityconsumption2020/keyfindings/#:~:text=The%20percentage%20of%20metered%20electricity,hours%20between%202015%20and%202020.

## refer to DCMEC2020TBL1.xlsx, DCMEC2020TBL2.xlsx and MEC02.20221208T201208.csv
## the last of which is the updated version of DCMEC2020TBL2.xlsx

## Q1 

### Find a dataset on the energy consumption of conventional data center storage and publish it on the Ocean Market. 
### Then, compare the energy performance of Filecoin storage to that of conventional data centers. What can you conclude?

#### Find a dataset on the energy consumption of conventional data center storage and publish it on the Ocean Market: 
#### PUT LINK HERE?????

#### Compare the energy performance of Filecoin storage to that of conventional data centers. What can you conclude?

In [1]:
import pandas as pd

### LOOKING AT RETRIEVED DATASETS

In [100]:
df1 = pd.read_excel("DCMEC2020TBL1.xlsx", skiprows=2) # data from ireland, data is in Gigawatt Hours (gWh)
df1

,Year,Data Centre,Other Metered Customers,Total,% Data Centre
0,2015,1236,23364,24600,5
1,2016,1477,23879,25356,6
2,2017,1755,23970,25725,7
3,2018,2172,24558,26730,8
4,2019,2478,24027,26505,9
5,2020,3019,24037,27056,11


In [7]:
df2 = pd.read_excel("DCMEC2020TBL2.xlsx", skiprows=2) #data is in Gigawatt Hours (gWh)
df2

,Year,Jan-Mar,Apr-Jun,Jul-Sep,Oct-Dec,Total
0,2015,290,302,315,328,1236
1,2016,339,359,384,394,1477
2,2017,405,432,448,470,1755
3,2018,488,524,571,589,2172
4,2019,576,598,635,669,2478
5,2020,688,719,763,849,3019


In [36]:
df3_preview = pd.read_csv("MEC02.20221208T201208.csv")
df3_preview.head()

,Statistic Label,Quarter,Electricity Consumption,UNIT,VALUE
0,Gigawatt hours,2015Q1,All metered electricity consumption,GWh,6565
1,Gigawatt hours,2015Q1,Data centres,GWh,290
2,Gigawatt hours,2015Q1,Customers other than data centres,GWh,6275
3,Gigawatt hours,2015Q2,All metered electricity consumption,GWh,6086
4,Gigawatt hours,2015Q2,Data centres,GWh,302


In [31]:
df3 = pd.read_csv("MEC02.20221208T201208.csv").drop(["Statistic Label", "UNIT"], axis=1, inplace=False)
df3

,Quarter,Electricity Consumption,VALUE
0,2015Q1,All metered electricity consumption,6565
1,2015Q1,Data centres,290
2,2015Q1,Customers other than data centres,6275
3,2015Q2,All metered electricity consumption,6086
4,2015Q2,Data centres,302
...,...,...,...
79,2021Q3,Data centres,1032
80,2021Q3,Customers other than data centres,5748
81,2021Q4,All metered electricity consumption,7158
82,2021Q4,Data centres,1058


In [38]:
df3_dc = df3[df3["Electricity Consumption"] == "Data centres"] # only data centres
df3_dc['Start Date'] = pd.to_datetime(df3_dc['Quarter']) # converting to 1st date in the quarter.
df3_dc.head()

/Users/limjohn/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Quarter,Electricity Consumption,VALUE,Start Date
1,2015Q1,Data centres,290,2015-01-01
4,2015Q2,Data centres,302,2015-04-01
7,2015Q3,Data centres,315,2015-07-01
10,2015Q4,Data centres,328,2015-10-01
13,2016Q1,Data centres,339,2016-01-01


In [35]:
# yearly consumption of electricity for Data Centres in Ireland
df3_dc.groupby(df3_dc['Start Date'].dt.to_period('Y'))['VALUE'].sum() 

Start Date
2015    1235
2016    1476
2017    1755
2018    2172
2019    2478
2020    3019
2021    3993
Freq: A-DEC, Name: VALUE, dtype: int64

### COMPARING ONLINE DATA OF ENERGY CONSUMPTION WITH FILECOIN STORAGE 

In [ ]:
# from https://www.akcp.com/blog/the-real-amount-of-energy-a-data-center-use/
## "worldwide data centers likely consumed roughly 205 terawatt-hours (TWh) in 2018, or 1% of global power use"
## from df1, DC electricity consumption: 8% of electricity consumption in Ireland (2018) to 11% of electricity consumption in Ireland (2020),
## as a result, we can expect some sort of increase in electricity consumption of DCs. 

## Assuming the rate of increase is exactly the same, 
## then we can assume that worldwide data centers would use about (11/8 * 1%) = 1.375% of global power use in 2020.
## If we were to extrapolate to 2021, assuming a constant increase in power use,
## then worldwide data centers would use about (11/8 * 1% * 3/2) = 2.0625% of global power use in 2021.

# from https://www.statista.com/statistics/280704/world-power-consumption/
## global power usage in 2021: 25343 tWh
## Hence, estimated Power Usage of worldwide data centers (in 2021) = 2.0625% * 25343 = 522.699 tWh

In [72]:
consum = pd.read_csv("FilecoinGreenEnergyConsumption-FilecoinGreenEnergyConsumption.csv", 
                          parse_dates = ['Date'])
consum["Date"] = consum["Date"].apply(lambda x: x.replace(tzinfo=None))


In [73]:
def get_date_range(df,start_date, end_date):
    mask = (df['Date'] > start_date) & (df['Date'] <= end_date)
    return mask

In [93]:
# finding the total amount of energy consumed per year (2020 and 2021)
cum_energy_use_2020 = consum[get_date_range(consum,"2020-01-01","2020-12-31")]["Cumulative energy use estimate"] 
cum_energy_use_2021 = consum[get_date_range(consum,"2021-01-01","2021-12-31")]["Cumulative energy use estimate"].reset_index(drop=True) 

# value on last day of year minus value of 1st day
energy_used_2020 = cum_energy_use_2020[-1:].values[0] - cum_energy_use_2020[0] 
energy_used_2021 = cum_energy_use_2021[-1:].values[0] - cum_energy_use_2021[0]

In [98]:
# total amount of energy consumed in 2020 in tWh
energy_used_2020/(10**9)

0.1281143749887

In [99]:
#  total amount of energy consumed in 2020 in tWh
energy_used_2021/(10**9)

1.1980393755

In [103]:
# finding the total amount of energy consumed per year (2020 and 2021)
cum_energy_use_2020_upper = consum[get_date_range(consum,"2020-01-01","2020-12-31")]["Cumulative energy use upper bound"] 
cum_energy_use_2021_upper = consum[get_date_range(consum,"2021-01-01","2021-12-31")]["Cumulative energy use upper bound"].reset_index(drop=True) 

# value on last day of year minus value of 1st day
energy_used_2020_upper = cum_energy_use_2020_upper[-1:].values[0] - cum_energy_use_2020_upper[0] 
energy_used_2021_upper = cum_energy_use_2021_upper[-1:].values[0] - cum_energy_use_2021_upper[0]

In [105]:
# Comparing energy consumption of Filecoin Network & Global Data Centres
print(f"Total Energy Used in 2021 for global data centres: 522.699 tWh") 
print(f"Total Energy Used in 2021 for Filecoin storage: {energy_used_2021/(10**9)} tWh") 
print(f"Total Energy Used (upper bound estimate) in 2021 for Filecoin storage: {energy_used_2021_upper/(10**9)} tWh") 

Total Energy Used in 2021 for global data centres: 522.699 tWh
Total Energy Used in 2021 for Filecoin storage: 1.1980393755 tWh
Total Energy Used (upper bound estimate) in 2021 for Filecoin storage: 2.9087399095 tWh


### Comparing the energy performance of Filecoin storage to that of conventional data centers, 
### there is a significant amount of energy saving using Filecoin storage as compared to traditional data centers, 
### even if we look at the upper bound of the energy consumption, as seen above. 
### There is an approx 520 tWh of energy savings in the year of 2021. 